In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from tqdm.auto import tqdm
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import TFBertModel
from transformers import BertTokenizer

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
file_path = '/content/gdrive/MyDrive/CSE438/labeled_data.csv'
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [7]:
row, col = df.shape
print('There are {} rows and {} columns'.format(row, col))

There are 24783 rows and 7 columns


In [8]:
c=df['class']
df.rename(columns={'tweet' : 'text',
                   'class' : 'category'}, 
                    inplace=True)
a=df['text']
b=df['category'].map({0: 'hate_speech', 1: 'offensive_language', 2: 'neither'})
df= pd.concat([a,b,c], axis=1)
df.rename(columns={'class' : 'label'}, 
                    inplace=True)

hate, offensive, neither = np.bincount(df['label'])
total = hate + offensive + neither
print('Total: {}'.format(total))
print('Hate: {} ({:.2f}% of total)\n'.format(hate, 100 * hate / total))
print('Offensive: {} ({:.2f}% of total)\n'.format(offensive, 100 * offensive / total))
print('Neither: {} ({:.2f}% of total)\n'.format(neither, 100 * neither / total))

Total: 24783
Hate: 1430 (5.77% of total)

Offensive: 19190 (77.43% of total)

Neither: 4163 (16.80% of total)



In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [10]:
token = tokenizer.encode_plus(
    df['text'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [11]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [12]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [13]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [14]:
labels = np.zeros((len(df), 5))
df.head()

,text,category,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,neither,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive_language,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive_language,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive_language,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive_language,1


In [15]:
labels[np.arange(len(df)), df['label'].values] = 1

In [16]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [17]:

def MapFunction(input_ids, attn_masks, labels):
    return {'input_ids': input_ids,'attention_mask': attn_masks}, labels


In [18]:

dataset = dataset.map(MapFunction)  

In [19]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) 

In [20]:
train_size = int((len(df)//16)*0.8)
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [21]:
from transformers import TFBertModel

In [22]:
model = TFBertModel.from_pretrained('bert-base-cased')  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [23]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] 
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(intermediate_layer)

model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accuracy = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [25]:
model.compile(optimizer=optimizer, loss=loss_func, metrics=[accuracy])

In [26]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
)

Epoch 1/5
1238/1238 [==============================] - 408s 299ms/step - loss: 0.3213 - accuracy: 0.8855 - val_loss: 0.1995 - val_accuracy: 0.9335
Epoch 2/5
1238/1238 [==============================] - 355s 287ms/step - loss: 0.2234 - accuracy: 0.9179 - val_loss: 0.1690 - val_accuracy: 0.9433
Epoch 3/5
1238/1238 [==============================] - 355s 287ms/step - loss: 0.1854 - accuracy: 0.9325 - val_loss: 0.1388 - val_accuracy: 0.9516
Epoch 4/5
1238/1238 [==============================] - 370s 299ms/step - loss: 0.1520 - accuracy: 0.9434 - val_loss: 0.0940 - val_accuracy: 0.9694
Epoch 5/5
1238/1238 [==============================] - 356s 288ms/step - loss: 0.1126 - accuracy: 0.9590 - val_loss: 0.0659 - val_accuracy: 0.9810


In [27]:
model.save('tweet_sentiment_model')

In [ ]:
sentiment_model = tf.keras.models.load_model('tweet_sentiment_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['hate_speech', 'offensive_language', 'neither']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:

input_text = input('Enter movie tweet: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

Enter movie review here: " Keeks is a bitch she curves everyone " lol I walked into a conversation like this. Smh
1/1 [==============================] - 3s 3s/step
Predicted Sentiment: offensive_language
